In [1]:
!pip3 install owlready2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from owlready2 import *

onto = get_ontology("http://test.org/productkg")
onto.base_iri

'http://test.org/productkg#'

In [3]:
from owlready2 import *

# Create the ontology
onto = get_ontology("http://www.semanticweb.org/german_tourism_activities")

with onto:
    # =========================================================================
    # 1. CLASSES (Concepts)
    # PascalCase for Classes
    # Metadata (rdfs:comment) added for definitions
    # =========================================================================

    class Activity(Thing):
        comment = ["A general concept for an activity of interest. Represents both physical venues and events."]
        label = ["Activity"]

    class City(Thing):
        comment = ["A German city where an activity is located."]
        label = ["City"]

    class BudgetTier(Thing):
        comment = ["A classification representing the cost of an Activity (e.g., Free, Low, High)."]
        label = ["Budget Tier"]

    class LocationSetting(Thing):
        # REPLACED WeatherSuitability with LocationSetting as requested
        comment = ["A classification of the environment where an activity takes place (e.g., Indoor, Outdoor)."]
        label = ["Location Setting"]

    class OperatingHours(Thing):
        comment = ["A concept to model a set of opening/closing times for specific days."]
        label = ["Operating Hours"]

    class DayOfWeek(Thing):
        comment = ["A specific day of the week (e.g., Monday, Sunday)."]
        label = ["Day of Week"]

    class Duration(Thing):
        comment = ["The length of time a tour takes."]
        label = ["Duration"]

    class Language(Thing):
        comment = ["A human language in which a tour is offered."]
        label = ["Language"]

    class MeetingPoint(Thing):
        comment = ["A physical location where a tour starts."]
        label = ["Meeting Point"]

    # --- Subclasses of Activity ---
    # Green arrows in your diagram denote Subclass relationships
    
    class Tour(Activity):
        comment = ["A guided activity to visit the city. It is an event, not a static location."]
        label = ["Tour"]

    class PhysicalVenue(Activity):
        comment = ["A physical location with set operating hours that people can visit."]
        label = ["Physical Venue"]

    # Disjointness: A Tour cannot be a Physical Venue
    AllDisjoint([Tour, PhysicalVenue])

    # --- Subclasses of PhysicalVenue ---

    class Museum(PhysicalVenue):
        comment = ["An institution displaying artifacts of cultural, artistic, or scientific interest."]
        label = ["Museum"]

    class Park(PhysicalVenue):
        comment = ["A public green space or natural area."]
        label = ["Park"]

    class NightlifeVenue(PhysicalVenue):
        comment = ["A location for evening entertainment, such as a bar or club."]
        label = ["Nightlife Venue"]

    class Sight(PhysicalVenue):
        comment = ["A location or structure of historical or architectural interest."]
        label = ["Sight"]

    # Venues are mutually exclusive
    AllDisjoint([Museum, Park, NightlifeVenue, Sight])

    # --- Classification Types ---

    class MuseumType(Thing):
        comment = ["Category of museum (e.g., Art, History)."]
        label = ["Museum Type"]
    
    class ParkType(Thing):
        comment = ["Category of park (e.g., Botanical Garden)."]
        label = ["Park Type"]

    class ClubType(Thing):
        comment = ["Category of nightlife venue (e.g., Disco, Bar)."]
        label = ["Club Type"]

    class SightType(Thing):
        comment = ["Category of sight (e.g., Monument)."]
        label = ["Sight Type"]

    # General Disjointness to prevent logical errors (e.g., a City cannot be a Duration)
    AllDisjoint([Activity, City, BudgetTier, LocationSetting, OperatingHours, 
                 DayOfWeek, Duration, Language, MeetingPoint])

    # =========================================================================
    # 2. OBJECT PROPERTIES (Relations)
    # CamelCase for Properties
    # FunctionalProperty used for 1:1 relations
    # =========================================================================

    class hasBudget(ObjectProperty, FunctionalProperty):
        # Functional: An activity belongs to exactly one budget tier
        domain = [Activity]
        range = [BudgetTier]
        comment = ["Connects an activity to its cost classification."]
        label = ["has budget"]

    class isInCity(ObjectProperty, FunctionalProperty):
        # Functional: An activity is located in one specific city.
        domain = [Activity]
        range = [City]
        comment = ["Connects an activity to the city it is located in."]
        label = ["is in city"]
    
    class isIndoorOrOutdoor(ObjectProperty):
        # REPLACED isSuitableFor. 
        # Not Functional: An activity could theoretically be both (e.g. combined venue).
        domain = [Activity]
        range = [LocationSetting]
        comment = ["Connects an activity to its location setting (Indoor or Outdoor)."]
        label = ["is indoor or outdoor"]

    # --- Tour Properties ---

    class hasDuration(ObjectProperty, FunctionalProperty):
        domain = [Tour]
        range = [Duration]
        comment = ["Specifies the duration of a tour."]
        label = ["has duration"]

    class hasLanguage(ObjectProperty):
        # Not Functional: Tours can be in multiple languages.
        domain = [Tour]
        range = [Language]
        comment = ["Specifies the languages available for a tour."]
        label = ["has language"]

    class hasMeetingPoint(ObjectProperty, FunctionalProperty):
        domain = [Tour]
        range = [MeetingPoint]
        comment = ["Specifies the starting location of a tour."]
        label = ["has meeting point"]
    
    # --- Venue Properties ---

    class hasOperatingHours(ObjectProperty):
        domain = [PhysicalVenue]
        range = [OperatingHours]
        comment = ["Connects a venue to its operating hours definition."]
        label = ["has operating hours"]

    class appliesToDay(ObjectProperty):
        domain = [OperatingHours]
        range = [DayOfWeek]
        comment = ["Specifies which day of the week these hours apply to."]
        label = ["applies to day"]

    # --- Classification Properties ---

    class hasMuseumType(ObjectProperty, FunctionalProperty):
        domain = [Museum]
        range = [MuseumType]
        comment = ["Classifies the museum."]
        label = ["has museum type"]
    
    class hasParkType(ObjectProperty, FunctionalProperty):
        domain = [Park]
        range = [ParkType]
        comment = ["Classifies the park."]
        label = ["has park type"]

    class hasClubType(ObjectProperty, FunctionalProperty):
        domain = [NightlifeVenue]
        range = [ClubType]
        comment = ["Classifies the nightlife venue."]
        label = ["has club type"]

    class hasSightType(ObjectProperty, FunctionalProperty):
        domain = [Sight]
        range = [SightType]
        comment = ["Classifies the sight."]
        label = ["has sight type"]

    # =========================================================================
    # 3. DATA PROPERTIES (Attributes)
    # OWL distinguishes Object vs Datatype properties
    # =========================================================================

    class opensAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [str] 
        comment = ["The time the venue opens (e.g. '09:00')."]
        label = ["opens at"]

    class closesAt(DataProperty, FunctionalProperty):
        domain = [OperatingHours]
        range = [str] 
        comment = ["The time the venue closes (e.g. '18:00')."]
        label = ["closes at"]

# Save the ontology
onto.save(file="german_city_tourism.owl", format="rdfxml")
print("Ontology created with LocationSetting.")

Ontology created with LocationSetting.
